<a href="https://colab.research.google.com/github/Zmuirhead03/Human-Behavior-Recognition-/blob/main/Human_Behavior_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import kagglehub
import os
import torch
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from PIL import Image

In [ ]:
datasetPath = kagglehub.dataset_download("abdullaalriad/stanford-40-full")
print("Dataset Path:", datasetPath)
print(os.listdir(datasetPath))


Using Colab cache for faster access to the 'stanford-40-full' dataset.
Dataset Path: /kaggle/input/stanford-40-full
['train_FUll']


In [ ]:
rootPath = datasetPath
dataRoot = os.path.join(rootPath, "train_FUll")
print("Data Root:", dataRoot)

Data Root: /kaggle/input/stanford-40-full/train_FUll


In [ ]:


transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std =[0.229, 0.224, 0.225])
])



In [ ]:
fullDataset = datasets.ImageFolder(root=dataRoot, transform=transform)
print("Number of images:", len(fullDataset))
print("Number of classes:", len(fullDataset.classes))
print("Classes (first 10):", fullDataset.classes[:10])

trainRatio = 0.8
trainSize = int(trainRatio * len(fullDataset))
valSize = len(fullDataset) - trainSize

trainDataset, valDataset = random_split(fullDataset, [trainSize, valSize])

batchSize = 32
trainLoader = DataLoader(trainDataset, batchSize, shuffle=True)
valLoader = DataLoader(valDataset, batchSize, shuffle=False)

print("Train Images:", len(trainDataset))
print("Val Images:", len(valDataset))

Number of images: 9532
Number of classes: 40
Classes (first 10): ['applauding', 'blowing_bubbles', 'brushing_teeth', 'cleaning_the_floor', 'climbing', 'cooking', 'cutting_trees', 'cutting_vegetables', 'drinking', 'feeding_a_horse']
Train Images: 7625
Val Images: 1907


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cuda


In [ ]:

numClasses = len(fullDataset.classes)

model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
inFeatures = model.fc.in_features
model.fc = nn.Linear(inFeatures, numClasses)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)


In [ ]:
numEpochs = 8

for epoch in range(numEpochs):

    model.train()
    runningLoss = 0.0
    runningCorrect = 0
    runningTotal = 0

    for images, labels in trainLoader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        runningLoss += loss.item() * images.size(0)

        _, preds = torch.max(outputs, 1)
        runningCorrect += torch.sum(preds == labels).item()
        runningTotal += labels.size(0)

    trainLoss = runningLoss / float(runningTotal)
    trainAcc = runningCorrect / float(runningTotal)

    model.eval()
    valLossSum = 0.0
    valCorrect = 0
    valTotal = 0

    with torch.no_grad():
        for images, labels in valLoader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            valLossSum += loss.item() * images.size(0)

            _, preds = torch.max(outputs, 1)
            valCorrect += torch.sum(preds == labels).item()
            valTotal += labels.size(0)

    valLoss = valLossSum / float(valTotal)
    valAcc = valCorrect / float(valTotal)

    print(
        "Epoch {}/{} | Train Loss {:.4f} Acc {:.4f} | Val Loss {:.4f} Acc {:.4f}".format(
            epoch + 1,
            numEpochs,
            trainLoss,
            trainAcc,
            valLoss,
            valAcc
        )
    )

Epoch 1/8 | Train Loss 1.7845 Acc 0.5690 | Val Loss 1.0764 Acc 0.7074
Epoch 2/8 | Train Loss 0.6316 Acc 0.8586 | Val Loss 0.9015 Acc 0.7493
Epoch 3/8 | Train Loss 0.2019 Acc 0.9744 | Val Loss 0.8301 Acc 0.7698
Epoch 4/8 | Train Loss 0.0531 Acc 0.9991 | Val Loss 0.8151 Acc 0.7787
Epoch 5/8 | Train Loss 0.0220 Acc 1.0000 | Val Loss 0.8031 Acc 0.7808
Epoch 6/8 | Train Loss 0.0130 Acc 0.9999 | Val Loss 0.7954 Acc 0.7829
Epoch 7/8 | Train Loss 0.0099 Acc 0.9996 | Val Loss 0.8395 Acc 0.7698
Epoch 8/8 | Train Loss 0.0070 Acc 1.0000 | Val Loss 0.8145 Acc 0.7845


In [ ]:
modelSavePath = "/content/stanford40_resnet18.pth"
torch.save(model.state_dict(), modelSavePath)
print("Model saved to:", modelSavePath)


Model saved to: /content/stanford40_resnet18.pth


In [ ]:
imagesFolder = "/content/behaviorTetsImages"
modelPath = "/content/stanford40_resnet18.pth"

classNames = [
    'applauding', 'blowing_bubbles', 'brushing_teeth', 'cleaning_the_floor',
    'climbing', 'cooking', 'cutting_trees', 'cutting_vegetables',
    'drinking', 'feeding_a_horse', 'fishing', 'fixing_a_bike',
    'fixing_a_car', 'gardening', 'holding_an_umbrella',
    'jumping', 'looking_through_a_microscope',
    'looking_through_a_telescope', 'phoning', 'playing_guitar',
    'playing_violin', 'pouring_liquid', 'pushing_a_cart', 'reading',
    'riding_a_bike', 'riding_a_horse', 'rowing_a_boat', 'running',
    'shooting_an_arrow', 'smoking', 'taking_photos', 'texting_message',
    'throwing_frisby', 'using_a_computer', 'walking_the_dog',
    'washing_dishes', 'watching_TV', 'waving_hands',
    'writing_on_a_board', 'writing_on_a_book'
]

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std =[0.229, 0.224, 0.225])
])

numClasses = len(classNames)

model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
inFeatures = model.fc.in_features
model.fc = nn.Linear(inFeatures, numClasses)
model.load_state_dict(torch.load(modelPath, map_location=device))

model = model.to(device)
model.eval()

print("Model loaded.")

validExts = [".jpg", ".jpeg", ".png", ".bmp"]

for filename in sorted(os.listdir(imagesFolder)):
    lowerName = filename.lower()
    isImage = False

    for ext in validExts:
        if lowerName.endswith(ext):
            isImage = True

    if not isImage:
        continue

    imgPath = os.path.join(imagesFolder, filename)
    img = Image.open(imgPath).convert("RGB")

    x = transform(img)
    x = x.unsqueeze(0).to(device)

    with torch.no_grad():
        outputs = model(x)
        _, predIdx = torch.max(outputs, 1)
        predIdx = predIdx.item()
        predLabel = classNames[predIdx]

    print(filename, "->", predLabel)

Using device: cuda
Model loaded.
applauding_009.jpg -> applauding
brushing_teeth_110.jpg -> brushing_teeth
cooking_217.jpg -> cooking
cutting_vegetables_009.jpg -> cutting_vegetables
fishing_011.jpg -> fishing
holding_an_umbrella_008.jpg -> holding_an_umbrella
phoning_005.jpg -> phoning
shooting_an_arrow_006.jpg -> shooting_an_arrow
waving_hands_005.jpg -> waving_hands
writing_on_a_board_037.jpg -> writing_on_a_board
